<a href="https://colab.research.google.com/github/Magouo/watki/blob/master/Lista4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import os
import time

def evaluate_chunk(indeksy, rozwiazanie, wagi, wartosci,
                   waga_biezaca, wartosc_biezaca, pojemnosc,
                   tabu_do_iteracji, indeks_iteracji,
                   najlepsza_wartosc_globalna, ignoruj_tabu):

    najlepszy_indeks = None
    najlepsza_wartosc = -10**9
    najlepsza_waga = None

    for i in indeksy:
        bit = rozwiazanie[i]
        delta_wagi = wagi[i] if bit == 0 else -wagi[i]
        nowa_waga = waga_biezaca + delta_wagi
        if nowa_waga > pojemnosc:
            continue

        delta_wartosci = wartosci[i] if bit == 0 else -wartosci[i]
        wartosc_kandydata = wartosc_biezaca + delta_wartosci

        if not ignoruj_tabu:
            jest_tabu = tabu_do_iteracji[i] > indeks_iteracji
            if jest_tabu and wartosc_kandydata <= najlepsza_wartosc_globalna:
                continue

        if (wartosc_kandydata > najlepsza_wartosc or
            (wartosc_kandydata == najlepsza_wartosc and
             (najlepsza_waga is None or nowa_waga < najlepsza_waga))):
            najlepszy_indeks = i
            najlepsza_wartosc = wartosc_kandydata
            najlepsza_waga = nowa_waga

    return najlepszy_indeks, najlepsza_wartosc, najlepsza_waga


def make_chunks(n, czesci):
    czesci = max(1, min(czesci, n))
    rozmiar = (n + czesci - 1) // czesci
    chunks = []
    start = 0
    while start < n:
        end = min(start + rozmiar, n)
        chunks.append(list(range(start, end)))
        start = end
    return chunks


class TabuSearchKnapsackParallel:
    def __init__(self, wagi, wartosci, pojemnosc,
                 maks_iteracji=2000,
                 kadencja_tabu=7,
                 rozrzut_kadencji=3,
                 cierpliwosc=200,
                 liczba_pracownikow=3,
                 ziarno=42):
        self.wagi = wagi
        self.wartosci = wartosci
        self.pojemnosc = pojemnosc
        self.maks_iteracji = maks_iteracji
        self.kadencja_tabu = kadencja_tabu
        self.rozrzut_kadencji = rozrzut_kadencji
        self.cierpliwosc = cierpliwosc
        self.liczba_pracownikow = liczba_pracownikow
        self.ziarno = ziarno

    def greedy_init(self):
        # start zachłanny
        n = len(self.wagi)
        kolejnosc = list(range(n))
        kolejnosc.sort(key=lambda i: self.wartosci[i] / self.wagi[i], reverse=True)
        rozwiazanie = [0] * n
        waga = 0
        wartosc = 0
        for i in kolejnosc:
            waga_i, wartosc_i = self.wagi[i], self.wartosci[i]
            if waga + waga_i <= self.pojemnosc:
                rozwiazanie[i] = 1
                waga += waga_i
                wartosc += wartosc_i
        return rozwiazanie, wartosc, waga

    def solve(self):
        n = len(self.wagi)
        los = random.Random(self.ziarno)

        rozwiazanie, wartosc_biezaca, waga_biezaca = self.greedy_init()
        najlepsze_rozwiazanie = list(rozwiazanie)
        najlepsza_wartosc = wartosc_biezaca
        najlepsza_waga = waga_biezaca

        tabu_do_iteracji = [0] * n
        bez_poprawy = 0
        czas_start = time.time()
        kawalki = make_chunks(n, self.liczba_pracownikow)

        with ThreadPoolExecutor(max_workers=self.liczba_pracownikow) as pula:
            for iteracja in range(1, self.maks_iteracji + 1):

                # 1) równoległa ocena sąsiedztwa z tabu
                zadania = [
                    pula.submit(
                        evaluate_chunk,
                        kawalek, rozwiazanie, self.wagi, self.wartosci,
                        waga_biezaca, wartosc_biezaca, self.pojemnosc,
                        tabu_do_iteracji, iteracja, najlepsza_wartosc, False
                    )
                    for kawalek in kawalki
                ]

                najlepszy_indeks = None
                najlepsza_wartosc_kandydata = -10**9
                najlepsza_waga_kandydata = None
                #menadzer zadan
                for z in as_completed(zadania):
                    indeks, wartosc_kandydata, waga_kandydata = z.result()
                    if indeks is None:
                        continue
                    if (wartosc_kandydata > najlepsza_wartosc_kandydata or
                        (wartosc_kandydata == najlepsza_wartosc_kandydata and
                         (najlepsza_waga_kandydata is None or
                          (waga_kandydata is not None and waga_kandydata < najlepsza_waga_kandydata)))):
                        najlepszy_indeks = indeks
                        najlepsza_wartosc_kandydata = wartosc_kandydata
                        najlepsza_waga_kandydata = waga_kandydata

                # 2) pomaga unikniecia w utknieciu programu przez deadlock
                if najlepszy_indeks is None:
                    zadania = [
                        pula.submit(
                            evaluate_chunk,
                            kawalek, rozwiazanie, self.wagi, self.wartosci,
                            waga_biezaca, wartosc_biezaca, self.pojemnosc,
                            tabu_do_iteracji, iteracja, najlepsza_wartosc, True
                        )
                        for kawalek in kawalki
                    ]
                    for z in as_completed(zadania):
                        indeks, wartosc_kandydata, waga_kandydata = z.result()
                        if indeks is None:
                            continue
                        if (wartosc_kandydata > najlepsza_wartosc_kandydata or
                            (wartosc_kandydata == najlepsza_wartosc_kandydata and
                             (najlepsza_waga_kandydata is None or
                              (waga_kandydata is not None and waga_kandydata < najlepsza_waga_kandydata)))):
                            najlepszy_indeks = indeks
                            najlepsza_wartosc_kandydata = wartosc_kandydata
                            najlepsza_waga_kandydata = waga_kandydata

                if najlepszy_indeks is None:
                    break

                # 3) wykonaj ruch
                if rozwiazanie[najlepszy_indeks] == 0:
                    delta_wagi = self.wagi[najlepszy_indeks]
                    delta_wartosci = self.wartosci[najlepszy_indeks]
                else:
                    delta_wagi = -self.wagi[najlepszy_indeks]
                    delta_wartosci = -self.wartosci[najlepszy_indeks]

                rozwiazanie[najlepszy_indeks] = 1
                waga_biezaca += delta_wagi
                wartosc_biezaca += delta_wartosci

                # 4) aktualizacja tabu
                kadencja = self.kadencja_tabu + (los.randint(0, self.rozrzut_kadencji) if self.rozrzut_kadencji > 0 else 0)
                tabu_do_iteracji[najlepszy_indeks] = iteracja + kadencja

                # 5) aktualizacja najlepszego
                if wartosc_biezaca > najlepsza_wartosc:
                    najlepsza_wartosc = wartosc_biezaca
                    najlepsza_waga = waga_biezaca
                    najlepsze_rozwiazanie = list(rozwiazanie)
                    bez_poprawy = 0
                else:
                    bez_poprawy += 1
                    if bez_poprawy >= self.cierpliwosc:
                        break

        return {
            "najlepsza_wartosc": najlepsza_wartosc,
            "najlepsza_waga": najlepsza_waga,
            "najlepsze_rozwiazanie": najlepsze_rozwiazanie,
        }

In [11]:
import cupy as cp

def evaluate_neighborhood_gpu(rozwiazanie_gpu, wagi_gpu, wartosci_gpu,
                              pojemnosc, tabu_gpu, idx_iteracji,
                              najlepsza_wartosc_globalna, ignoruj_tabu):

    waga_biezaca = int(cp.sum(rozwiazanie_gpu * wagi_gpu).get())
    wartosc_biezaca = int(cp.sum(rozwiazanie_gpu * wartosci_gpu).get())

    delta_wagi = cp.where(rozwiazanie_gpu == 0, wagi_gpu, -wagi_gpu)
    delta_wartosci = cp.where(rozwiazanie_gpu == 0, wartosci_gpu, -wartosci_gpu)

    nowa_waga = waga_biezaca + delta_wagi
    wartosc_kandydata = wartosc_biezaca + delta_wartosci

    maska_feasible = nowa_waga <= pojemnosc

    if ignoruj_tabu:
        maska_tabu_ok = cp.ones_like(maska_feasible, dtype=cp.bool_)
    else:
        jest_tabu = tabu_gpu > idx_iteracji
        maska_aspiracja = wartosc_kandydata > najlepsza_wartosc_globalna
        maska_tabu_ok = cp.logical_not(jest_tabu) | maska_aspiracja

    maska_ok = maska_feasible & maska_tabu_ok

    if not bool(maska_ok.any().get()):
        return None, None, None, waga_biezaca, wartosc_biezaca

    minus_inf = cp.min(wartosc_kandydata) - 10_000
    wartosc_przefiltrowana = cp.where(maska_ok, wartosc_kandydata, minus_inf)

    idx_best = int(cp.argmax(wartosc_przefiltrowana).get())
    najlepsza_wartosc = int(wartosc_kandydata[idx_best].get())
    najlepsza_waga = int(nowa_waga[idx_best].get())

    return idx_best, najlepsza_wartosc, najlepsza_waga, waga_biezaca, wartosc_biezaca


class TabuSearchKnapsackGPU:
    def __init__(self, wagi, wartosci, pojemnosc,
                 maks_iteracji=2000,
                 kadencja_tabu=7,
                 rozrzut_kadencji=3,
                 ziarno=42):
        self.wagi_cpu = wagi
        self.wartosci_cpu = wartosci
        self.pojemnosc = pojemnosc
        self.maks_iteracji = maks_iteracji
        self.kadencja_tabu = kadencja_tabu
        self.rozrzut_kadencji = rozrzut_kadencji
        self.ziarno = ziarno

        self.wagi_gpu = cp.array(wagi, dtype=cp.int32)
        self.wartosci_gpu = cp.array(wartosci, dtype=cp.int32)

    def greedy_init(self):
        n = len(self.wagi_cpu)
        stosunek = [self.wartosci_cpu[i] / self.wagi_cpu[i] for i in range(n)]
        kolejnosc = sorted(range(n), key=lambda i: stosunek[i], reverse=True)
        rozwiazanie = [0] * n
        waga = 0
        wartosc = 0
        for i in kolejnosc:
            w_i, v_i = self.wagi_cpu[i], self.wartosci_cpu[i]
            if waga + w_i <= self.pojemnosc:
                rozwiazanie[i] = 1
                waga += w_i
                wartosc += v_i
        return rozwiazanie, wartosc, waga

    def solve(self):
        los = random.Random(self.ziarno)

        rozwiazanie_cpu, wartosc_biezaca, waga_biezaca = self.greedy_init()
        rozwiazanie_gpu = cp.array(rozwiazanie_cpu, dtype=cp.int8)

        najlepsze_rozwiazanie_gpu = rozwiazanie_gpu.copy()
        najlepsza_wartosc = wartosc_biezaca
        najlepsza_waga = waga_biezaca

        tabu_gpu = cp.zeros(len(self.wagi_cpu), dtype=cp.int32)
        czas_start = time.time()

        for iteracja in range(1, self.maks_iteracji + 1):

            idx_best, wartosc_kand, waga_kand, waga_biezaca, wartosc_biezaca = evaluate_neighborhood_gpu(
                rozwiazanie_gpu, self.wagi_gpu, self.wartosci_gpu, self.pojemnosc,
                tabu_gpu, iteracja, najlepsza_wartosc, ignoruj_tabu=False
            )

            if idx_best is None:
                idx_best, wartosc_kand, waga_kand, waga_biezaca, wartosc_biezaca = evaluate_neighborhood_gpu(
                    rozwiazanie_gpu, self.wagi_gpu, self.wartosci_gpu, self.pojemnosc,
                    tabu_gpu, iteracja, najlepsza_wartosc, ignoruj_tabu=True
                )

            if idx_best is None:
                break

            rozwiazanie_gpu[idx_best] = 1 - rozwiazanie_gpu[idx_best]

            kadencja = self.kadencja_tabu + (los.randint(0, self.rozrzut_kadencji) if self.rozrzut_kadencji > 0 else 0)
            tabu_gpu[idx_best] = iteracja + kadencja

            waga_biezaca = waga_kand
            wartosc_biezaca = wartosc_kand

            if wartosc_biezaca > najlepsza_wartosc:
                najlepsza_wartosc = wartosc_biezaca
                najlepsza_waga = waga_biezaca
                najlepsze_rozwiazanie_gpu = rozwiazanie_gpu.copy()

        najlepsze_rozwiazanie_cpu = najlepsze_rozwiazanie_gpu.get().tolist()
        return {
            "najlepsza_wartosc": najlepsza_wartosc,
            "najlepsza_waga": najlepsza_waga,
            "najlepsze_rozwiazanie": najlepsze_rozwiazanie_cpu,
        }

In [ ]:
import matplotlib.pyplot as plt

def test_cpu_gpu(n_przedmiotow):
    los = random.Random(123)
    wagi = [los.randint(100, 500) for _ in range(n_przedmiotow)]
    wartosci = [los.randint(10, 200) for _ in range(n_przedmiotow)]
    pojemnosc = int(0.4 * sum(wagi))

    # CPU
    solver_cpu = TabuSearchKnapsackParallel(
        wagi, wartosci, pojemnosc,
        maks_iteracji=2000, kadencja_tabu=7, rozrzut_kadencji=3, cierpliwosc=300,
        liczba_pracownikow=max(1, (os.cpu_count() or 2)), ziarno=42
    )
    t0 = time.time()
    wynik_cpu = solver_cpu.solve()
    t1 = time.time()
    czas_cpu = t1 - t0

    # GPU
    solver_gpu = TabuSearchKnapsackGPU(
        wagi, wartosci, pojemnosc,
        maks_iteracji=2000, kadencja_tabu=7, rozrzut_kadencji=3, ziarno=42
    )
    t2 = time.time()
    wynik_gpu = solver_gpu.solve()
    cp.cuda.Stream.null.synchronize()
    t3 = time.time()
    czas_gpu = t3 - t2

    return czas_cpu, czas_gpu, wynik_cpu["najlepsza_wartosc"], wynik_gpu["najlepsza_wartosc"]


rozmiary = [500, 1000, 1500, 2000,20000,100000]

czasy_cpu = []
czasy_gpu = []
wartosci_cpu = []
wartosci_gpu = []

for n in rozmiary:
    print(f"\n=== n = {n} ===")
    c_cpu, c_gpu, v_cpu, v_gpu = test_cpu_gpu(n)
    czasy_cpu.append(c_cpu)
    czasy_gpu.append(c_gpu)
    wartosci_cpu.append(v_cpu)
    wartosci_gpu.append(v_gpu)
    print(f"CPU: czas = {c_cpu:.4f} s, wartość = {v_cpu}")
    print(f"GPU: czas = {c_gpu:.4f} s, wartość = {v_gpu}")

# 1) czas CPU vs GPU
plt.figure(figsize=(7,5))
plt.plot(rozmiary, czasy_cpu, marker='o', label="CPU (ThreadPool)")
plt.plot(rozmiary, czasy_gpu, marker='s', label="GPU (CuPy)")
plt.xlabel("Liczba przedmiotów (n)")
plt.ylabel("Czas [s]")
plt.title("Czas wykonania Tabu Search: CPU vs GPU")
plt.legend()
plt.grid(True)
plt.show()

# 2) speedup
speedup = [czasy_cpu[i] / czasy_gpu[i] if czasy_gpu[i] > 0 else 0 for i in range(len(rozmiary))]

plt.figure(figsize=(7,5))
plt.plot(rozmiary, speedup, marker='^', color='purple', label="Speedup (CPU czas / GPU czas)")
plt.xlabel("Liczba przedmiotów (n)")
plt.ylabel("Przyspieszenie (CPU czas / GPU czas)")
plt.title("Przyspieszenie GPU względem CPU")
plt.grid(True)
plt.axhline(1.0, color='red', linestyle='--', label="Brak przyspieszenia (1x)")
plt.legend()
plt.show()


=== n = 500 ===
CPU: czas = 0.2777 s, wartość = 45292
GPU: czas = 2.9035 s, wartość = 34862

=== n = 1000 ===
CPU: czas = 0.6579 s, wartość = 94019
GPU: czas = 2.4079 s, wartość = 71157

=== n = 1500 ===
CPU: czas = 0.9140 s, wartość = 139631
GPU: czas = 2.4214 s, wartość = 106806

=== n = 2000 ===
CPU: czas = 1.0545 s, wartość = 178288
GPU: czas = 2.3573 s, wartość = 140728

=== n = 20000 ===
CPU: czas = 7.3220 s, wartość = 1428375
GPU: czas = 2.3945 s, wartość = 1404596

=== n = 100000 ===
